# BETA_random_eqrthquake_locations_w_pyvista_vtk

Testing the random picking of subsurface eqrthquake locations (moment-tensor locations) and 3D plotting of the picks and model 

### Step 0

Load packages

In [ ]:
#load all packages
from sys import argv
import numpy as np
import pyvista as pv
import pickle
import matplotlib.pyplot as plt 
from matplotlib.colors import Normalize
from pyaspect.model.gridmod3d import gridmod3d as gm
from pyaspect.model.bbox import bbox as bb
from pyaspect.model.gm3d_utils import compress_gm3d_to_file
from pyaspect.model.gm3d_utils import decompress_gm3d_from_file
import copy

### Step 1 

Extract the ndarray of the subsampled, smoothed NAM model and instantiate a new GriddedModel3D object for QC'ing

In [ ]:
data_in_dir  = 'data/output/'
data_out_dir = data_in_dir
!ls {data_in_dir} 

### Step 6 

Decompress the ndarray of the sliced, subsampled, smoothed NAM model and instantiate a new GriddedModel3D object for QC'ing

In [ ]:
# set filename then used it to decompress model
ifqn = f'{data_out_dir}/vsliced_subsmp_smth_nam_2017_vp_vs_rho_Q_model_dx100_dy100_dz100_maxdepth5850_sig250.npz'
vslice_gm3d, other_pars = decompress_gm3d_from_file(ifqn)

print()
print('decompressed gridded model\n:',vslice_gm3d) 
print()
print('other parameters:\n',other_pars)
print()

# WARNING: this will unpack all other_pars, if you overwrite a variable of the samename as val(key), then you 
#          may not notice, and this may cause large headaches.  I use it because I am aware of it.
'''
for key in other_pars:
    locals()[key] = other_pars[key]  #this is more advanced python than I think is reasonable for most 
sig_meters = sig
''';

# another way to get these varibles is just use the accessor functions for the gridmod3d.  We need them later.
xmin = other_pars['xmin']
dx   = other_pars['dx']
nx   = other_pars['nx']
ymin = other_pars['ymin']
dy   = other_pars['dy']
ny   = other_pars['ny']
zmin = other_pars['zmin']
dz   = other_pars['dz']
nz   = other_pars['nz']
sig_meters = other_pars['sig']  # this variable is used later
print('sig_meters:',sig_meters)

In [ ]:
# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape + 1 because we want to inject our values on
#   the CELL data
nam_dims = list(vslice_gm3d.get_npoints())
nam_origin = list(vslice_gm3d.get_gorigin())
nam_origin[2] *= -1
nam_origin = tuple(nam_origin)
nam_spacing = list(vslice_gm3d.get_deltas())
nam_spacing[2] *=-1
nam_spacing = tuple(nam_spacing)
print('nam_dims:',nam_dims)
print('nam_origin:',nam_origin)
print('nam_spacing:',nam_spacing)

# Edit the spatial reference
grid.dimensions = np.array(nam_dims) + 1
grid.origin = nam_origin  # The bottom left corner of the data set
grid.spacing = nam_spacing  # These are the cell sizes along each axis
nam_pvalues = vslice_gm3d.getNPArray()[0]
print('pvalues.shape:',nam_pvalues.shape)

# Add the data values to the cell data
grid.cell_arrays["values"] = nam_pvalues.flatten(order="F")  # Flatten the array!

# Now plot the grid!
cmap = plt.cm.jet
#grid.plot(show_edges=True,cmap=cmap)
grid.plot(cmap=cmap,opacity=1.0)
print('dir:\n',dir(grid))


In [ ]:
slices = grid.slice_orthogonal()

#slices.plot(show_edges=True,cmap=cmap)
slices.plot(cmap=cmap)

In [ ]:
print('vslice_gm32:\n',vslice_gm3d)
coords = vslice_gm3d.getGlobalCoordsPointsXYZ()
coords[:,2] = -coords[:,2]
print(coords.shape)
print('norm:\n',coords)

xc = vslice_gm3d.getLocalCoordsPointsX()
yc = vslice_gm3d.getLocalCoordsPointsY()
zc = vslice_gm3d.getLocalCoordsPointsZ()

thing = np.meshgrid(xc,yc,zc)
print('thing0.shape:',thing[0].shape)
print('thing1.shape:',thing[1].shape)
print('thing2.shape:',thing[2].shape)
print('thing0:\n',thing[0])
print('thing1:\n',thing[1])
print('thing2:\n',thing[2])

'''
print(coords.shape)
coords = coords.transpose()
#coords[0,:] += np.arange(4)*100000
#coords[1,:] -= np.arange(4)*100000
print('transp:\n',coords)
sort_coords = coords [ :, coords[2].argsort()]
coords = sort_coords
print('sort:\n',coords)
coords = coords.transpose()
print('norm:\n',coords)
coords = coords[::4,:]
''';
def skip_by_index(arr,i,skip):
    #print('arr norm-shape:\n',arr.shape)
    arr = arr.transpose()
    print('arr trans:\n',arr)
    sort_arr = arr[ :, arr[i].argsort()]
    arr = sort_arr
    arr = arr.transpose()
    arr = arr[::skip,:]
    return arr

def get_unique_xyz_skip(arr,skip):
    ux = np.unique(arr.T[0,:])
    print('nx:',len(np.unique(arr.T[0,:])))
    uy = np.unique(arr.T[1,:])
    print('ny:',len(np.unique(arr.T[1,:])))
    uz = np.unique(arr.T[2,:])
    print('nz:',len(np.unique(arr.T[2,:])))
    
    ux = ux[::skip]
    uy = uy[::skip]
    uz = uz[::skip]
    
    return np.vstack(np.meshgrid(ux,uy,uz)).reshape(3,-1).T
   
coords = get_unique_xyz_skip(coords,8)



#coords = skip_by_index(coords,2,8)
#coords = skip_by_index(coords,1,8)
#coords = skip_by_index(coords,0,8)

print('coords after sort:\n',coords)
print('coords after shape:\n',coords.shape)


'''
coords = coords[::4,:]
print()
print(coords.shape)
print(coords)
coords[:,2] = -coords[:,2]
print(coords)
coords = coords.transpose()
print('transp:\n',coords)
sort_coords = coords [ :, coords[2].argsort()]
coords = sort_coords
coords = coords[:,::4].transpose()
sort_coords = coords [ :, coords[1].argsort()]
coords = sort_coords
coords = coords[:,::4].transpose()
print('sort:\n',coords)
''';

In [ ]:
pv_points = pv.wrap(coords)
p = pv.Plotter()
#p.add_mesh(slices,cmap=cmap)
p.add_mesh(grid,cmap=cmap,opacity=0.85)
p.add_mesh(pv_points, render_points_as_spheres=True, point_size=10,opacity=0.85)
p.show()

In [ ]:
coords = vslice_gm3d.getGlobalCoordsPointsXYZ()
coords[:,2] = -coords[:,2]

xc = np.unique(coords.T[0,:])
yc = np.unique(coords.T[1,:])
zc = np.unique(coords.T[2,:])

#print('nx:\n',len(xc))
#print('x:\n',xc)


n_rx = 20
n_ry = 20
n_rz = 20

lrx = np.min(xc)
lry = np.min(yc)
lrz = -4300.0

hrx = np.max(xc)
hry = np.max(yc)
hrz = -3000.0

srx = hrx - lrx
sry = hry - lry
srz = hrz - lrz

r_xyz_list = []
for i in range(n_rx*n_ry*n_rz):
    rx = lrx + srx*np.random.rand()
    ry = lry + sry*np.random.rand()
    rz = lrz + srz*np.random.rand()
    r_xyz_list.append([rx,ry,rz])
    
r_xyz = np.array(r_xyz_list)
    

#rx = lrx + srx*np.random.rand(n_rx)
#ry = lry + sry*np.random.rand(n_ry)
#rz = lrz + srz*np.random.rand(n_rz)

#print('rx:\n',rx)
#print('ry:\n',ry)
#print('rz:\n',rz)

#r_xyz = np.vstack(np.meshgrid(rx,ry,rz)).reshape(3,-1).T
print('r_xyz:\n',r_xyz)


In [ ]:
pv_rpoints = pv.wrap(r_xyz)
p = pv.Plotter()
#p.add_mesh(slices,cmap=cmap,opacity=0.50)
p.add_mesh(grid,cmap=cmap,opacity=0.75)
p.add_mesh(pv_rpoints, render_points_as_spheres=True, point_size=5,opacity=0.5)
p.show()

In [ ]:
print(len(r_xyz))

In [ ]:
!pwd

In [ ]:
#!mkdir {data_out_dir}/tmp
!ls {data_out_dir}

In [ ]:
import os
import copy
import pickle

from time import time

from functools import total_ordering
from functools import wraps


def timer(func):
    @wraps(func)
    def wrap(*args, **kwargs):
        t_start = time()
        result = func(*args, **kwargs)
        t_end = time()
        print(f'Function \'{func.__name__}()\' executed in {(t_end-t_start):4.3f}s\n')
        return result
    return wrap


#@total_ordering
class Station:
    
    def __init__(self, name=None, network=None, lat_yc=None, lon_xc=None, elevation=None, burial=None, trid=0):
        
        self.name      = name
        self.network   = network
        self.lat_yc    = lat_yc
        self.lon_xc    = lon_xc
        self.elevation = elevation
        self.burial    = burial
        self.trid       = trid
        
        if 32 < len(self.name):
            raise Exception('Station.name cannot exceed 32 characters')
        
        
    def __str__(self):
        '''
        return str({'name':self.name,
                    'network':self.network,
                    'lat_yc':self.lat_yc,
                    'lon_xc':self.lon_xc,
                    'elevation':self.elevation,
                    'burial':self.burial,
                    'trid':self.trid})
        ''';
        return str(self.name)
    
    def __repr__(self):
        return str({'name':self.name,
                    'network':self.network,
                    'lat_yc':self.lat_yc,
                    'lon_xc':self.lon_xc,
                    'elevation':self.elevation,
                    'burial':self.burial,
                    'trid':self.trid})
        '''
        return str(self.name)
        ''';
                    
    def __hash__(self):
        '''
        return hash(str({'network':self.network,
                         'lat_yc':self.lat_yc,
                         'lon_xc':self.lon_xc,
                         'elevation':self.elevation,
                         'burial':self.burial}))
        if self.name == 'dummy':
            h = hash(np.sqrt(self.lon_xc**2 + self.lat_yc**2 + self.burial**2))
            print(f'{self.name}.hash:{hash(h)}')
        if self.name == 't0220zm':
            h = hash(np.sqrt(self.lon_xc**2 + self.lat_yc**2 + self.burial**2))
            print(f'{self.name}.hash:{hash(h)}')
        ''';
        return hash(np.sqrt(self.lon_xc**2 + self.lat_yc**2 + self.burial**2))
    
    def _is_valid_operand(self, other):
        return (hasattr(other, 'network')    and
                hasattr(other, 'lat_yc')     and
                hasattr(other, 'lon_xc')     and
                hasattr(other, 'elevation')  and
                hasattr(other, 'burial')     and
                hasattr(other, 'trid')) 
    
    def __eq__(self, other):
        if not self._is_valid_operand(other):
            return NotImplemented
        return ((self.lat_yc, self.lon_xc, self.elevation, self.burial) ==
                (other.lat_yc, other.lon_xc, other.elevation, other.burial))
    
    def __ne__(self, other):
        if not self._is_valid_operand(other):
            return NotImplemented
        return ((self.lat_yc, self.lon_xc, self.elevation, self.burial) !=
                (other.lat_yc, other.lon_xc, other.elevation, other.burial))
        
    def __lt__(self, other):
        #print(f'{self.name}.{self.trid} < {other.name}.{other.trid}')
        if not self._is_valid_operand(other):
            return NotImplemented
        return self.trid < other.trid
        
    def __le__(self, other):
        if not self._is_valid_operand(other):
            return NotImplemented
        return (self.trid <= other.trid)
        
    def __gt__(self, other):
        if not self._is_valid_operand(other):
            return NotImplemented
        return self.trid > other.trid
    
    def __ge__(self, other):
        if not self._is_valid_operand(other):
            return NotImplemented
        return (self.trid >= other.trid)
        

    
@timer
def write_from_station_list(fqp,stations):
    
    path = os.path.relpath(fqp, start=os.curdir)
    fqpname = os.path.join(path, 'STATIONS')
    header_fqpname = os.path.join(path, 'pyheader.stations')
    
    print('fqpname:', fqpname)

    str_stations = []
    for i in range(len(stations)):
        s = stations[i]
        sname = f'{s.name}_{s.trid}_{s.gid}_{s.sid}'
        snet  = s.network
        slat  = s.lat_yc # or Y coordinate
        slon  = s.lon_xc # or X coordinate
        selev = s.elevation
        sbur  = s.burial
        str_stations.append('%s %s %.2f %.2f %.2f %.2f\n' %(sname,snet,slat,slon,selev,sbur))

    f = open(fqpname, 'w')
    f.writelines(str_stations)
    f.close()
    
    f = open(header_fqpname, 'wb')
    pickle.dump(stations,f)
    f.close()
    
@timer
def read_station_list(fqp):
    
    path = os.path.relpath(fqp, start=os.curdir)
    header_fqpname = os.path.join(path, 'pyheader.stations')
    
    f = open(header_fqpname, 'rb')
    stations = pickle.load(f)
    f.close()
    
    return stations
    

def make_station_group(station=None,delta=None):
    
    station_group = []
    
    cpy_station = copy.deepcopy(station)
    cpy_station.gid = station.gid = 0
    station_group.append(cpy_station)
    
    # add x - delta
    station_xm = copy.deepcopy(station)
    station_xm.lon_xc -= delta
    station_xm.gid  = 1
    station_group.append(station_xm)
    
    # add x + delta
    station_xp = copy.deepcopy(station)
    station_xp.lon_xc += delta
    station_xp.gid  = 2
    station_group.append(station_xp)
    
    # add y - delta
    station_ym = copy.deepcopy(station)
    station_ym.lat_yc -= delta
    station_ym.gid  = 3
    station_group.append(station_ym)
    
    # add y + delta
    station_yp = copy.deepcopy(station)
    station_yp.lat_yc += delta
    station_yp.gid  = 4
    station_group.append(station_yp)
    
    # add z - delta
    station_zm = copy.deepcopy(station)
    station_zm.burial -= delta
    station_zm.gid  = 5
    station_group.append(station_zm)
    
    # add y + delta
    station_zp = copy.deepcopy(station)
    station_zp.burial += delta
    station_zp.gid  = 6
    station_group.append(station_zp)
    
    return station_group
    
    
@timer
def make_station_group_list(stations,delta):
    
    
    group_station_list = []
    for i in range(len(stations)):
        group_station = make_station_group(station=stations[i],delta=delta)
        #group_station_list = list(set(group_station_list + group_station))
        group_station_list += group_station
    
    return list(set(group_station_list))
    

l_stations = []

for i in range(len(r_xyz)):
    
    name = 't' + str(i).zfill(len(str(len(r_xyz))))
    new_s = Station(name=name,
                    network='NL',
                    lat_yc=r_xyz[i,1],
                    lon_xc=r_xyz[i,0],
                    elevation=0.0,
                    burial=r_xyz[i,2],
                    trid=i)
    l_stations.append(new_s)
                                           
l_grp_stations = make_station_group_list(stations=l_stations,delta=5.0)
ts = copy.deepcopy(l_grp_stations[-1])
ts1 = copy.deepcopy(l_grp_stations[-1])
ts2 = copy.deepcopy(l_grp_stations[-1])
ts2.trid = -2
print('ts1:\n',ts1)
print('ts2:\n',ts2)
print('ts1 == ts2:',ts1 != ts2)
mylist = [ts1] + [ts2]
myset = sorted(set(mylist))
print('myset:\n',myset)
#ts.trid = -2
#ts.name = 'dummy'
print(ts)
l_grp_stations += [ts]
#l_grp_stations = l_stations

In [ ]:
#print(l_grp_stations)
print('len:',len(l_grp_stations))
l_grp_stations = set(l_grp_stations)
#l_grp_stations = sorted(set(l_grp_stations))
print('len:',len(l_grp_stations))
for s in sorted(l_grp_stations):
    print(s.name)
#write_stations(data_out_dir + '/tmp',l_stations)
#write_stations(data_out_dir + '/tmp',l_grp_stations)
#write_from_station_list(data_out_dir + '/tmp',l_grp_stations)

In [ ]:
class Header(dict):
    
    def __init__(self):
        super(Header,self).__init__()
        
        
    def __getattr__(self, key):
        if name in self:
            return self[key]
        else:
            raise AttributeError(f'Header: {key} not found' )

    def __setattr__(self, key, value):
        self[key] = value

    def __delattr__(self, key):
        if key in self:
            del self[key]
        else:
            raise AttributeError(f'Header: {key} not found' )
            
    def __hash__(self):
        return hash(self.hash_val())
    
    def _is_valid_operand(self, other):
        return self.keys() == other.keys()
    
    def __eq__(self, other):
        if not self._is_valid_operand(other):
            return NotImplemented
        return (self.eq_comparator() == other.eq_comparator())
    
    def __ne__(self, other):
        if not self._is_valid_operand(other):
            return NotImplemented
        return (self.eq_comparator() != other.eq_comparator())
        
    def __lt__(self, other):
        if not self._is_valid_operand(other):
            return NotImplemented
        return self.comparator() < other.comparator()
        
    def __le__(self, other):
        if not self._is_valid_operand(other):
            return NotImplemented
        return self.comparator() <= other.comparator()
        
    def __gt__(self, other):
        if not self._is_valid_operand(other):
            return NotImplemented
        return self.comparator() > other.comparator()
    
    def __ge__(self, other):
        if not self._is_valid_operand(other):
            return NotImplemented
        return self.comparator() >= other.comparator()
        
        
    def hash_val(self):
        raise NotImplementedError
        
    def comparator(self):
        raise NotImplementedError
        
    def eq_comparator(self):
        raise NotImplementedError
            
            
class StationHeader(Header):
    
    def __init__(self, 
                 name=None,
                 network=None,
                 lat_yc=None,
                 lon_xc=None,
                 elevation=None,
                 burial=None,
                 trid=0,
                 gid=0,
                 sid=0):
        
        super(StationHeader,self).__init__()
        
        if 32 < len(name):
            raise Exception('Station.name cannot exceed 32 characters')
        
        self['name']      = name
        self['network']   = network
        self['lat_yc']    = lat_yc
        self['lon_xc']    = lon_xc
        self['elevation'] = elevation
        self['burial']    = burial
        self['trid']      = trid
        self['gid']       = gid
        self['sid']       = sid
        
        self['comp_val']  = self.trid
        
        self['ids'] = {}
        
    
    def hash_val(self):
        return np.sqrt(self.lon_xc**2 + self.lat_yc**2 + self.burial**2)
        
    def comparator(self):
        return self.comp_val
        
    def eq_comparator(self):
        return (self.lat_yc, self.lon_xc, self.elevation, self.burial)

    
    @property
    def comp_val(self):
        return self['comp_val']

    @comp_val.setter
    def comp_val(self, key, value):
        self[key] = value

    
    @property
    def name(self):
        return self['name']

    @name.setter
    def name(self, value):
        self['name'] = value


    @property
    def network(self):
        return self['network']

    @network.setter
    def network(self, value):
        self['network'] = value


    @property
    def lat_yc(self):
        return self['lat_yc']

    @lat_yc.setter
    def lat_yc(self, value):
        self['lat_yc'] = value


    @property
    def lon_xc(self):
        return self['lon_xc']

    @lon_xc.setter
    def lon_xc(self, value):
        self['lon_xc'] = value


    @property
    def elevation(self):
        return self['elevation']

    @elevation.setter
    def elevation(self, value):
        self['elevation'] = value


    @property
    def burial(self):
        return self['burial']

    @burial.setter
    def burial(self, value):
        self['burial'] = value


    @property
    def trid(self):
        return self['trid']

    @trid.setter
    def trid(self, value):
        self['trid'] = value


    @property
    def gid(self):
        return self['gid']

    @gid.setter
    def gid(self, value):
        self['gid'] = value
        

    @property
    def sid(self):
        return self['sid']

    @sid.setter
    def sid(self, value):
        self['sid'] = value
        
        
    @property
    def ids(self):
        return self['ids']


In [ ]:
l_stations = []

for i in range(len(r_xyz)):
    
    name = 't' + str(i).zfill(len(str(len(r_xyz))))
    new_s = StationHeader(name=name,
                          network='NL',
                          lat_yc=r_xyz[i,1],
                          lon_xc=r_xyz[i,0],
                          elevation=0.0,
                          burial=r_xyz[i,2],
                          trid=i)
    l_stations.append(new_s)
                                           
l_grp_stations = make_station_group_list(stations=l_stations,delta=5.0)
ts = copy.deepcopy(l_grp_stations[-1])
#ts.trid = -2
ts.name = 'dummy'
ts.ids['goobid'] = 13
print('ts:',ts)
l_grp_stations += [ts]
#l_grp_stations = l_stations

In [ ]:
#print(l_grp_stations)
print('len:',len(l_grp_stations))
#l_grp_stations = set(l_grp_stations)
l_grp_stations = sorted(set(l_grp_stations))
print('dummy.hash:31880328642150367')
print('len:',len(l_grp_stations))
#for s in sorted(l_grp_stations):
    #print(s.name)
#write_stations(data_out_dir + '/tmp',l_stations)
#write_stations(data_out_dir + '/tmp',l_grp_stations)
write_from_station_list(data_out_dir + '/tmp',l_grp_stations)

In [ ]:
!tail ../data/output/tmp/STATIONS

In [ ]:
!ls {data_out_dir + '/tmp'} 

In [ ]:
fqp = data_out_dir + '/tmp'
rw_stations = sorted(read_station_list(fqp))

for s in sorted(rw_stations):
    print(f'{s.name}_{s.trid}_{s.gid}_{s.sid}')

In [ ]:
import os

path = os.path.relpath(data_out_dir + '///', start=os.curdir)
newp = os.path.join(path, 'tmp', 'STATIONS')

print(newp)


In [ ]:
!ls /Users/seismac/Documents/Work/Bench/ForkGnam/pyaspect/notebooks/Full_Workflow/FWI_Workflow_for_Groningen
